In [73]:
import sys,os
sys.path.append(os.getcwd())
import utflow
import functools

import logging
logging.basicConfig(level=logging.DEBUG)

## Basic structure of task definition and execution.

In [74]:
flow = utflow.TaskFlow()

In [75]:
with flow.next("1st") as task:
    with task.start() as _:
        _.x = 5
        _.y = 3
        print("1st: ", _.x, _.y)
        # context '_' is saved as pickle file in '.taskflow' directory after task execution.

with flow.next("2nd") as task:
    with task.start() as _:
        print("2nd: %s,%s"%(_.x, _.y))
        _.x = 10
        _.z = 20

with flow.next("last") as task:
    with task.start() as _:
        print("3rd: x=%s"%_.x)
        print("3rd: z=%s"%_.z)

DEBUG:utflow.taskflow:Dump state of task '1st'
DEBUG:utflow.taskflow:Dump state of task '2nd'
DEBUG:utflow.taskflow:Dump state of task 'last'


1st:  5 3
2nd: 5,3
3rd: x=10
3rd: z=20


## Continue from '2nd' task

In [76]:
flow.reset(start="2nd")

In [77]:
with flow.next("1st") as task:
    with task.start() as _:
        _.x = 5
        _.y = 3
        print("1st: ", _.x, _.y)

with flow.next("2nd") as task:
    with task.start() as _:
        print("2nd: %s,%s"%(_.x, _.y))
        _.x = 10
        _.z = 20

with flow.next("last") as task:
    with task.start() as _:
        print("3rd: x=%s"%_.x)
        print("3rd: z=%s"%_.z)

DEBUG:utflow.taskflow: Skipped task '1st'
DEBUG:utflow.taskflow:Load state of previous task '1st'
DEBUG:utflow.taskflow:Dump state of task '2nd'
DEBUG:utflow.taskflow:Dump state of task 'last'


2nd: 10,3
3rd: x=10
3rd: z=20


## Continue from 'last' task 

In [78]:
flow.reset(start="last")

In [79]:
with flow.next("1st") as task:
    with task.start() as _:
        _.x = 5
        _.y = 3
        print("1st: ", _.x, _.y)

with flow.next("2nd") as task:
    with task.start() as _:
        print("2nd: %s,%s"%(_.x, _.y))
        _.x = 10
        _.z = 20

with flow.next("last") as task:
    with task.start() as _:
        print("3rd: x=%s"%_.x)
        print("3rd: z=%s"%_.z)

DEBUG:utflow.taskflow: Skipped task '1st'
DEBUG:utflow.taskflow: Skipped task '2nd'
DEBUG:utflow.taskflow:Load state of previous task '2nd'
DEBUG:utflow.taskflow:Dump state of task 'last'


3rd: x=10
3rd: z=20


## Specifying 'end' task

In [80]:
flow.reset(end="2nd")

In [81]:
with flow.next("1st") as task:
    with task.start() as _:
        _.x = 5
        _.y = 3
        print("1st: ", _.x, _.y)

with flow.next("2nd") as task:
    with task.start() as _:
        print("2nd: %s,%s"%(_.x, _.y))
        _.x = 10
        _.z = 20

with flow.next("last") as task:
    with task.start() as _:
        print("3rd: x=%s"%_.x)
        print("3rd: z=%s"%_.z)

DEBUG:utflow.taskflow:Load state of previous task 'last'
DEBUG:utflow.taskflow:Dump state of task '1st'
DEBUG:utflow.taskflow: Skipped task '2nd'
DEBUG:utflow.taskflow: Skipped task 'last'


1st:  5 3


## Conditional execution of Task

In [82]:
flow.reset()

In [83]:
with flow.next("1st") as task:
    with task.start() as _:
        _.x = 5
        _.y = 3
        print("1st: ", _.x, _.y)

# Task is executed if 'x' in context is less than 5.
with flow.next("2nd", when=lambda _: _.x < 5) as task:
    with task.start() as _:
        print("2nd -- x<5")
        print("2nd: %s,%s"%(_.x, _.y))
        _.x = 1
        _.z = 2

# Task is executed if 'x' in context is greater than or equal to 5
with flow.next("2nd", when=lambda _: _.x >=5) as task:
    with task.start() as _:
        print("2nd -- x>=5")
        print("2nd: %s,%s"%(_.x, _.y))
        _.x = 9
        _.z = 4

with flow.next("last") as task:
    with task.start() as _:
        print("3rd: x=%s"%_.x)
        print("3rd: z=%s"%_.z)

DEBUG:utflow.taskflow:Load state of previous task 'last'
DEBUG:utflow.taskflow:Dump state of task '1st'
DEBUG:utflow.taskflow:Duplicate state of task '1st' as '2nd'
DEBUG:utflow.taskflow: Conditionally skipped task '2nd'
DEBUG:utflow.taskflow:Dump state of task '2nd'
DEBUG:utflow.taskflow:Dump state of task 'last'


1st:  5 3
2nd -- x>=5
2nd: 5,3
3rd: x=9
3rd: z=4


## Substitution of context using '<<' operator

In [84]:
kwargs = {"test1": 1, "test2": 2}
flow.reset()

state = utflow.TaskFlow.Task.Context()
state.xxx = "XXX"

with flow.next() as task:
    with task.start() as _:
        _ << kwargs << state
        print(_.test1)
        print(_.xxx)

DEBUG:utflow.taskflow:Load state of previous task 'last'
DEBUG:utflow.taskflow:Dump state of task '0'


1
XXX


## Pipe like interface for Context and functions using '>>' operator

In [85]:
def hello(greeting, name):
    text = "%s, %s"%(greeting, name)
    return {"text": text }

def to_list(text):
    return [text]

def print_text(string):
    print(string)
    return None

with flow.next() as task:
    with task.start() as _:
        
        _.greeting = "Hello"
        _.name     = "world"
        
        # Passing context to function as arguments, and get result wrapped by context.
        c = _ >> hello
        print(c.text)
        print()
        
        # If result of function call is dict, '>>' put all the keys to target context
        _ >> hello >> _
        print(_.text)
        print()
        
        # If result of function call is dict, '>>' select appropriate keys from dict, and put them as '**kwargs' in next function call.
        # If result of function call is list, '>>' pass all the result as '*args' in next function call.
        _ >> hello >> to_list >> print_text

DEBUG:utflow.taskflow:Dump state of task '1'


Hello, world

Hello, world

Hello, world
